In [102]:
import pandas as pd
import numpy as np

In [103]:
vdf = pd.read_csv('fruitvegprices-2017_2022.csv')

In [104]:
vdf

,category,item,variety,date,price,unit
0,fruit,apples,bramleys_seedling,2022-03-11,2.05,kg
1,fruit,apples,coxs_orange_group,2022-03-11,1.22,kg
2,fruit,apples,egremont_russet,2022-03-11,1.14,kg
3,fruit,apples,braeburn,2022-03-11,1.05,kg
4,fruit,apples,gala,2022-03-11,1.03,kg
...,...,...,...,...,...,...
9642,cut_flowers,alstromeria,indoor,2017-11-03,0.27,stem
9643,cut_flowers,chrysanthemum,indoor_spray,2017-11-03,0.22,stem
9644,cut_flowers,lillies,oriental,2017-11-03,0.70,stem
9645,cut_flowers,narcissus,indoor,2017-11-03,0.06,stem


## Ejercicios

### Ejercicio uno: realice un analisis de valores nulos

En caso de existir valores nulos elimine esos registros

In [26]:
vdf.isnull().sum()

category    0
item        0
variety     0
date        0
price       0
unit        0
dtype: int64

### Ejercicio dos: Por categoria, cuantos distintos items hay y cuantos distintos variety hay?

El resultado debe de tener el siguiente formato de presentacion:

In [26]:
pd.DataFrame({
    "category" : ['cut_flowers','fruit']
    , "item_count" : [9,10]
    , "variety_Count" : [10,21]
})

,category,item_count,variety_Count
0,cut_flowers,9,10
1,fruit,10,21


Como nota en un mismo dataframe mostramos todo lo solicitado, usted debe de mostrar todas las categorias

In [233]:
vdf.loc[:,['category','item','variety']]

,category,item,variety
0,fruit,apples,bramleys_seedling
1,fruit,apples,coxs_orange_group
2,fruit,apples,egremont_russet
3,fruit,apples,braeburn
4,fruit,apples,gala
...,...,...,...
9642,cut_flowers,alstromeria,indoor
9643,cut_flowers,chrysanthemum,indoor_spray
9644,cut_flowers,lillies,oriental
9645,cut_flowers,narcissus,indoor


In [25]:
vdf.groupby(['category',], as_index=False).agg({ 'variety':['count'],'item':['count']  })

,category,variety,item
,,count,count
0,cut_flowers,342,342
1,fruit,1992,1992
2,pot_plants,49,49
3,vegetable,7264,7264


### Ejercicio tres: Para la combinacion item, variety muestre el ultimo valor registrado por año.

Ejemplo de como debe de mostrar su respuesta

In [239]:
pd.DataFrame({
    "item" : ['Manzana']
    , "variety" : ['Gala']
    , "year" : [2022]
    , "last_price" :  [3.2]
})

,item,variety,year,last_price
0,Manzana,Gala,2022,3.2


In [240]:
vdf['date'] = pd.to_datetime(vdf['date'])

In [241]:
vdf['year'] = vdf['date'].dt.year

In [242]:
vdf

,category,item,variety,date,price,unit,year,Month,resultado,year-month
0,fruit,apples,bramleys_seedling,2022-03-11,2.05,kg,2022,3,fruit apples,2022-3
1,fruit,apples,coxs_orange_group,2022-03-11,1.22,kg,2022,3,fruit apples,2022-3
2,fruit,apples,egremont_russet,2022-03-11,1.14,kg,2022,3,fruit apples,2022-3
3,fruit,apples,braeburn,2022-03-11,1.05,kg,2022,3,fruit apples,2022-3
4,fruit,apples,gala,2022-03-11,1.03,kg,2022,3,fruit apples,2022-3
...,...,...,...,...,...,...,...,...,...,...
9642,cut_flowers,alstromeria,indoor,2017-11-03,0.27,stem,2017,11,cut_flowers alstromeria,2017-11
9643,cut_flowers,chrysanthemum,indoor_spray,2017-11-03,0.22,stem,2017,11,cut_flowers chrysanthemum,2017-11
9644,cut_flowers,lillies,oriental,2017-11-03,0.70,stem,2017,11,cut_flowers lillies,2017-11
9645,cut_flowers,narcissus,indoor,2017-11-03,0.06,stem,2017,11,cut_flowers narcissus,2017-11


In [227]:
m=vdf.groupby(['year'], as_index=False)

In [228]:
m

In [42]:
x=vdf.loc[:,['item','variety','year','price']]

In [43]:
x

,item,variety,year,price
0,apples,bramleys_seedling,2022,2.05
1,apples,coxs_orange_group,2022,1.22
2,apples,egremont_russet,2022,1.14
3,apples,braeburn,2022,1.05
4,apples,gala,2022,1.03
...,...,...,...,...
9642,alstromeria,indoor,2017,0.27
9643,chrysanthemum,indoor_spray,2017,0.22
9644,lillies,oriental,2017,0.70
9645,narcissus,indoor,2017,0.06


In [56]:
x=vdf.groupby(['item', 'variety','year'], as_index=False).agg({'price':['max'] })

In [57]:
x

,item,variety,year,price
,,,,max
0,alstromeria,indoor,2017,0.29
1,alstromeria,indoor,2018,0.34
2,apples,braeburn,2017,0.72
3,apples,braeburn,2018,1.05
4,apples,braeburn,2019,1.11
...,...,...,...,...
392,turnip,turnip,2021,1.83
393,turnip,turnip,2022,1.87
394,watercress,pillow_packs,2017,11.44


### Ejercicio cuatro: Segun el ejercicio tres, que tanto aumento o disminuyo el precio de cada item-variety 

Ejemplo de como se debe de mostrar su respuesta

In [28]:
pd.DataFrame({
    "item" : ['Manzana','Manzana','Manzana']
    , "variety" : ['Gala','Gala','Gala']
    , "year" : [2020,2021,2022]
    , "last_price" :  [2.7,2.5,3.2]
    , "delta" : [0, (2.5-2.7), (3.2-2.5)]
})

,item,variety,year,last_price,delta
0,Manzana,Gala,2020,2.7,0.0
1,Manzana,Gala,2021,2.5,-0.2
2,Manzana,Gala,2022,3.2,0.7


Segun nuestro ejemplo dummy para 2021 estuvo 0.2 centavos mas barato y en 2022 estuvo 0.7 centavos caro

In [70]:
y=vdf.loc[:,['item','variety','price','year']]

In [71]:
y

,item,variety,price,year
0,apples,bramleys_seedling,2.05,2022
1,apples,coxs_orange_group,1.22,2022
2,apples,egremont_russet,1.14,2022
3,apples,braeburn,1.05,2022
4,apples,gala,1.03,2022
...,...,...,...,...
9642,alstromeria,indoor,0.27,2017
9643,chrysanthemum,indoor_spray,0.22,2017
9644,lillies,oriental,0.70,2017
9645,narcissus,indoor,0.06,2017


In [76]:
y.groupby(['year','item','variety','price'], as_index=False)

In [77]:
y

,item,variety,price,year
0,apples,bramleys_seedling,2.05,2022
1,apples,coxs_orange_group,1.22,2022
2,apples,egremont_russet,1.14,2022
3,apples,braeburn,1.05,2022
4,apples,gala,1.03,2022
...,...,...,...,...
9642,alstromeria,indoor,0.27,2017
9643,chrysanthemum,indoor_spray,0.22,2017
9644,lillies,oriental,0.70,2017
9645,narcissus,indoor,0.06,2017


### Ejercicio cinco: Cual es precio promedio por item, variety

Ejemplo de como se debe de mostrar su respuesta

In [29]:
pd.DataFrame({
    "item" : ['Manzana']
    , "variety" : ['Gala']
    , "year-month" : ['2021-01']   
    , "avg-price" : [0.5]
})

,item,variety,year-month,avg-price
0,Manzana,Gala,2021-01,0.5


In [108]:
vdf['Month'] = vdf['date'].dt.month

In [109]:
vdf

,category,item,variety,date,price,unit,year,Month
0,fruit,apples,bramleys_seedling,2022-03-11,2.05,kg,2022,3
1,fruit,apples,coxs_orange_group,2022-03-11,1.22,kg,2022,3
2,fruit,apples,egremont_russet,2022-03-11,1.14,kg,2022,3
3,fruit,apples,braeburn,2022-03-11,1.05,kg,2022,3
4,fruit,apples,gala,2022-03-11,1.03,kg,2022,3
...,...,...,...,...,...,...,...,...
9642,cut_flowers,alstromeria,indoor,2017-11-03,0.27,stem,2017,11
9643,cut_flowers,chrysanthemum,indoor_spray,2017-11-03,0.22,stem,2017,11
9644,cut_flowers,lillies,oriental,2017-11-03,0.70,stem,2017,11
9645,cut_flowers,narcissus,indoor,2017-11-03,0.06,stem,2017,11


In [ ]:
vdf['resultado'] = vdf['category'].int.cat(vdf['item'], sep =" ")

In [158]:
vdf.dtypes


category             object
item                 object
variety              object
date         datetime64[ns]
price               float64
unit                 object
year                 object
Month                object
resultado            object
dtype: object

In [161]:
vdf['year-month'] = vdf['year'].str.cat(vdf['Month'], sep ="-")

In [163]:
vdf

,category,item,variety,date,price,unit,year,Month,resultado,year-month
0,fruit,apples,bramleys_seedling,2022-03-11,2.05,kg,2022,3,fruit apples,2022-3
1,fruit,apples,coxs_orange_group,2022-03-11,1.22,kg,2022,3,fruit apples,2022-3
2,fruit,apples,egremont_russet,2022-03-11,1.14,kg,2022,3,fruit apples,2022-3
3,fruit,apples,braeburn,2022-03-11,1.05,kg,2022,3,fruit apples,2022-3
4,fruit,apples,gala,2022-03-11,1.03,kg,2022,3,fruit apples,2022-3
...,...,...,...,...,...,...,...,...,...,...
9642,cut_flowers,alstromeria,indoor,2017-11-03,0.27,stem,2017,11,cut_flowers alstromeria,2017-11
9643,cut_flowers,chrysanthemum,indoor_spray,2017-11-03,0.22,stem,2017,11,cut_flowers chrysanthemum,2017-11
9644,cut_flowers,lillies,oriental,2017-11-03,0.70,stem,2017,11,cut_flowers lillies,2017-11
9645,cut_flowers,narcissus,indoor,2017-11-03,0.06,stem,2017,11,cut_flowers narcissus,2017-11


In [174]:
s=vdf.groupby(['item','variety','year-month'], as_index=False).agg({ 'price':['mean'] })

In [175]:
s.columns=s.columns.droplevel(1)

In [176]:
s.columns = ['item', 'variety', 'year-month', 'avg_price']

In [177]:
s

,item,variety,year-month,avg_price
0,alstromeria,indoor,2017-11,0.2700
1,alstromeria,indoor,2017-12,0.2800
2,alstromeria,indoor,2018-1,0.3100
3,alstromeria,indoor,2018-10,0.2350
4,alstromeria,indoor,2018-11,0.2580
...,...,...,...,...
2580,watercress,pillow_packs,2019-4,9.4750
2581,watercress,pillow_packs,2019-5,9.5000
2582,watercress,pillow_packs,2019-6,9.4900
2583,watercress,pillow_packs,2019-7,9.4975


### Ejercicio seis: muestre por category,item,variety que tanto % ha aumentado su precio

Ejemplo: si para las manzanas gala el primer precio que se registro fue 2.00 y el ultimo precio que se registro es 4.00 este aumento un 100%

In [36]:
pd.DataFrame({
    "category": ["fruta","fruta"]
    ,"item" : ['Manzana','Manzana']
    , "variety" : ['Gala','egremont_russet']
    , "percentage-change-between-first-and-last-price" : [200,150]       
})

,category,item,variety,percentage-change-between-first-and-last-price
0,fruta,Manzana,Gala,200
1,fruta,Manzana,egremont_russet,150


In [217]:
x=vdf.loc[:,['category','item','variety','date','price']]

In [218]:
x=x.groupby(['category','item','variety','price'], as_index=False)

In [219]:
x = x.agg(fecha_minima=('date', np.min), fecha_maxima=('date', np.max))

In [220]:
x

,category,item,variety,price,fecha_minima,fecha_maxima
0,cut_flowers,alstromeria,indoor,0.23,2018-07-06,2018-11-30
1,cut_flowers,alstromeria,indoor,0.24,2018-04-13,2018-09-21
2,cut_flowers,alstromeria,indoor,0.25,2018-08-24,2018-10-26
3,cut_flowers,alstromeria,indoor,0.26,2017-11-10,2018-11-16
4,cut_flowers,alstromeria,indoor,0.27,2017-11-03,2018-12-07
...,...,...,...,...,...,...
4152,vegetable,watercress,pillow_packs,11.29,2018-02-02,2018-02-02
4153,vegetable,watercress,pillow_packs,11.44,2017-12-08,2017-12-08
4154,vegetable,watercress,pillow_packs,11.90,2018-01-26,2018-01-26
4155,vegetable,watercress,pillow_packs,12.09,2018-03-23,2018-03-23


### Ejercicio siete: Segun el ejercicio seis por category cual es el item,variety con mayor aumento de crecimiento 

Segun el ejemplo dummy que se utilizo en el ejercicio seis para la category fruta se mostraria la manzana gala unicamente